# K shortest paths

For discovering the latest time one can leave to arrive not later than the specified time from one station to another, we use modified <b>k shortest path</b> algorithm. More specifically, our algorithm is a constrained shortest paths algorithm. All the possibilites we consider is that it must be a valid route on the the public transport infrastructure and schedule and arrive not later than the given time. Additionally, we tried to integrate threshold into the method as well as you can see, however we still left both options there for an integrated approach.

The algorithm itself in the k_shortest notebook starts from the last (target) station and adds trips from new stations at each step according to <b>G</b> making sure such connections are feasible in time and allowed 2 minutes of transfer whenever the trip_id changes. Additionally, we also allow 2 minute transfer time (mostly for getting out of the transport, since it usually takes time) when walking from one station to the other, in additional to walking time. We make sure we only keep 15 shortest paths from any source to the destination, thus limiting algorithm complexity and making it more feasible regarding its time asymptotics. 

We used ground truth given by TAs as well as manually generated samples using SBB wesbite and timetable. Our algorithm passed our tests and more details can be found in validation section of the notebook.

Content of the notebook:

0. Helper Functions
1. Parameters/ Variables
3. K shortest path algorithm
4. Validation/Evaluation

In [1]:
%%configure
{"conf": {
    "spark.app.name": "my-awesome-group_final",
    "spark.driver.memory": "2g"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8491,application_1589299642358_3026,pyspark,idle,Link,Link,
8521,application_1589299642358_3056,pyspark,idle,Link,Link,
8639,application_1589299642358_3164,pyspark,idle,Link,Link,
8642,application_1589299642358_3167,pyspark,idle,Link,Link,
8693,application_1589299642358_3218,pyspark,idle,Link,Link,
8695,application_1589299642358_3220,pyspark,idle,Link,Link,
8696,application_1589299642358_3221,pyspark,idle,Link,Link,
8697,application_1589299642358_3222,pyspark,idle,Link,Link,
8698,application_1589299642358_3223,pyspark,idle,Link,Link,
8700,application_1589299642358_3225,pyspark,idle,Link,Link,


In [2]:
# Initialization
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8730,application_1589299642358_3255,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Helper Functions

In [3]:
import pandas as pd 
from math import sin, cos, sqrt, atan2, radians
from geopy import distance as dist
from pyspark.sql.functions import col
from heapq import *
import math

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def compute_seconds(time_string):
    """
    Given time_string in the form of hh:mm:ss, computes total number of seconds
    """

    split = time_string.split(':')
    return int(split[0])*3600 + int(split[1])*60 + int(split[2])

assert compute_seconds('02:00:01') == 7201

def compute_string(seconds):
    """
    Given seconds returns string corresponding to the time of the day
    """
    hour = seconds // 3600
    minute = (seconds - hour*3600) // 60
    second = seconds % 60
    return '%02d:%02d:%02d'%(hour, minute, second)

assert compute_string(300) == '00:05:00' and compute_string(43503) == '12:05:03'

def compute_duration_between(time_string1, time_string2):
    """
    Computes how much time passed from time_string1 to time_string2
    """
    return compute_seconds(time_string2) - compute_seconds(time_string1)

assert compute_duration_between('08:00:00', '08:05:00') == 300

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# cell to communicate with hdfs
import subprocess, pickle

def run_cmd(args_list):
    """Run linux commands."""
    print('Running system command: {0}'.format(' '.join(args_list)))    
    proc = subprocess.Popen(args_list,                            
                            stdout=subprocess.PIPE,                            
                            stderr=subprocess.PIPE)    
    s_output, s_err = proc.communicate()    
    s_return =  proc.returncode
    return s_return, s_output, s_err


def save_hdfs(localPath, hdfsPath):
    
    (ret, out, err)= run_cmd(['hdfs','dfs','-put','-f', localPath, hdfsPath])
    if err:
        print(err)
    else:
        print('Success')
        
def read_hdfs(hdfsPath):
    
    (ret, out, err)= run_cmd(['hdfs','dfs','-cat', hdfsPath])
    if err:
        print(err)
    else:
        print('Success')
    return pickle.loads(out)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
class StationNode:
    """
    Class for station node, that will keep a list of arrival/departures given by trips as well
    as walking distances
    """
    
    def __init__(self, station_id):
    
        self.station_id = station_id
        self.departures = dict()
        self.arrivals = dict()  
        self.walkable_stations = dict()
 
    def add_walkable_station(self, stop_id, duration):
        """
        Add walkable station and duration to get there
        """
        if stop_id not in self.walkable_stations:
            self.walkable_stations[stop_id] = duration

    def add_arrival(self, time, arrival):
        """
        Arrivals are in the form of
        """
        if time not in self.arrivals:
            self.arrivals[time] = []
            
        self.arrivals[time].append(arrival)
        
    def add_departure(self, time, departure):
        """
        Departure are in the form of
        """
        if time not in self.departures:
            self.departures[time] = []
            
        self.departures[time].append(departure)
        
    def to_tuple(self):
        """
        Convert class object to tuple
        """
        return (self.station_id, self.departures, self.arrivals, self.walkable_stations)
    
    def from_tuple(self, class_tuple):
        """
        Read class fields from tuple
        """
        self.station_id = class_tuple[0]
        self.departures = class_tuple[1]
        self.arrivals = class_tuple[2]
        self.walkable_stations = class_tuple[3]
        
        return self
        
def dictnodes_tolist(station_nodes):
    """
    Convert dictionary of StationNodes to list of node tuples
    """
    tolist = []
    for stop_id in station_nodes:
        try:
            tolist.append(station_nodes[stop_id].to_tuple())
        except:
            raise Exception(stop_id)
        
    return tolist
        
def list_todictnodes(fromlist):
    """
    Convert list of node tuples into dictionary of StationNodes
    """
    todict = dict()
    for node_tuple in fromlist:
        todict[node_tuple[0]] = StationNode(node_tuple[0]).from_tuple(node_tuple)
        
    return todict

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Parameters/ Variables

In [7]:
G = read_hdfs('/user/lortkipa/graph_untested_classTuples.pkl')
G = list_todictnodes(G)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Running system command: hdfs dfs -cat /user/lortkipa/graph_untested_classTuples.pkl
Success

In [8]:
# stops and their names
stops = read_hdfs('/user/lortkipa/filtered_stops_Premoved.pkl')
stops = stops.set_index('stop_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Running system command: hdfs dfs -cat /user/lortkipa/filtered_stops_Premoved.pkl
Success

In [9]:
# starting stop
#start_stop = u'8503000'
start_stop = '8591060'
# ending stop
#end_stop = u'8591049'
end_stop = u'8503376'
#end_stop = '8591060'
# arrival_time
T = compute_seconds('12:30:00')
#T = compute_seconds('10:00:00')

# number of shortest paths to discover
K = 5


# disctionary for number of shortest paths found from each node
count_u = dict()

# SBB network used for the algorithm
sbb_network = G

# all possible nodes/stations
all_stops = G.keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## K-Shortest

In [10]:
from copy import deepcopy


def k_shortest(threshold=None, Pierre=None, Pierre2=None, remove_same=True):
    """
    Implementation of k shortest path given the threshold if threshold is not None
    K path starts from destinations and finds path to source backwards
    
    Pierre return probability of transfer given start_station, end_station, departure_time, arrival_time(latest)
    """
    # set of shortest paths from s to t
    P = []

    # containing path, maintaining heap data structure
    h_paths = [] 

    # initialize number of shortes path found to each node to 0
    for node in all_stops:
        count_u[node] = 0
        
    # min cost of the path
    min_cost = []

    # insert path from last stop with cost 0   
    # we have ( cost_of_path(time it lasts), 
    #  [(station, time_at_station_seconds, route/walk/terminus, duration, probability_of_success) ..])
    heappush(h_paths, (0, [(end_stop, T, 'Terminus', '00:00:00', 1)]))

    # while we have possible paths and found shortest paths from start+stop aren't already K
    while h_paths and count_u[start_stop] < K:

        C, shortes_path_T = heappop(h_paths)

        # read the first node and its corresponding time on shortest path
        first_node_on_path = shortes_path_T[-1][0]
        first_node_time = shortes_path_T[-1][1]
        trip = shortes_path_T[-1][2]
        succ_prob = shortes_path_T[-1][4]
        #print('stop', first_node_on_path,'time', first_node_time, start_stop)

        # update number of paths
        count_u[first_node_on_path] += 1

        if first_node_on_path == start_stop:
            if C in min_cost and remove_same:
                count_u[first_node_on_path] -= 1
                continue
            min_cost.append(C)
            P.append((C, shortes_path_T))

        if count_u[first_node_on_path] <= K:

            # return Node of the first stop on the path
            u_node = sbb_network[first_node_on_path]

            # iterate over stations that can reach the first stop on the path by transport
            for time_edge in u_node.arrivals:
                
                # get the time these stations arrive at the first stop of our path
                arrival_time = compute_seconds(time_edge)
                
                
                # traverse all the incoming stations for time_edge time
                for station_edge in u_node.arrivals[time_edge]:
                    
                    
                    # transfer time to the new trip (possible platform) of the same station, 0 if the same trip
                    transfer_time = 2 * 60 if station_edge[2] != trip and trip != 'Terminus' else 0
                    
                    # check if one can make with this station_edge
                    if  arrival_time > first_node_time - transfer_time:
                        continue
                        
                    updated_probe=1
                    # check that probability of making will still be above the threshold
                    if threshold is not None and station_edge[2] != trip:

                        
                        if trip == 'terminus':
                            # prev_stop_id, prev_stop_tripid, arrival_time, final_stop, final_arrival_time
                            n_proba = Pierre2(end_stop, 
                                              station_edge[2], 
                                              time_edge, 
                                              end_stop, 
                                              compute_string(T))
                        elif trip != 'walk':
                            # prev_stop_id, prev_stop_tripid, arrival_time, depart_stop, dep_trip, departure_time
                            n_proba = Pierre(first_node_on_path, 
                                             station_edge[2], 
                                             time_edge, 
                                             first_node_on_path,
                                             trip,
                                             compute_string(first_node_time))
                        else:
                            n_proba = Pierre(first_node_on_path, 
                                             station_edge[2], 
                                             time_edge, 
                                             shortes_path_T[-2][0],
                                             shortes_path_T[-2][2],
                                             compute_string(shortes_path_T[-2][1]))
                            
                        
                        # compute cumulated probability of success
                        if succ_prob * n_proba < threshold:
                            continue
                        
                        updated_probe = succ_prob * n_proba
                        
                    departure_time = compute_seconds(station_edge[1])

                    new_path = deepcopy(shortes_path_T)
                    new_path.append((station_edge[0], departure_time, station_edge[2], 
                                     compute_string(arrival_time-departure_time), updated_probe))

                    # insert new nodes into the heap
                    heappush(h_paths, 
                             (C + first_node_time - departure_time, 
                             new_path))  

            # iterate over walkable station, assume 2 mn delay for connection         
            for station_id in u_node.walkable_stations:
                # if it is not the same station and previous wasnt also walk
                if station_id != first_node_on_path and trip != 'walk':

                    new_path = deepcopy(shortes_path_T)
                    new_path.append((station_id, first_node_time - u_node.walkable_stations[station_id] - 2*60, 
                                     'walk', compute_string(u_node.walkable_stations[station_id]),succ_prob))

                    # insert new nodes into the heap
                    heappush(h_paths, 
                             (C + u_node.walkable_stations[station_id] + 2*60, 
                             new_path))  
    return P
                
answ = k_shortest(remove_same=False)

#8591049, 8591128, 8591830, 8590626

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# write just one value if the path is same:
def combine_same_trips(path):
    renewed_path = []
    for connection in path:
        #print(connection)
        if (len(renewed_path) > 0) and (renewed_path[-1][2] == connection[2]):
            renewed_path[-1] = (renewed_path[-1][0],
                                renewed_path[-1][1],
                                renewed_path[-1][2],
                                compute_string(compute_seconds(renewed_path[-1][3]) + compute_seconds(connection[3])))
        else:
            renewed_path.append(connection)
    return renewed_path


def print_answ(answ, is_print=True):
    refined = []
    for i,tp in enumerate(answ):
        path = deepcopy(tp[1])
        path.reverse()
        path = [(p[0], compute_string(p[1]), p[2], p[3]) for p in path]
        refined.append((tp[0]/60, combine_same_trips(path)))
        if is_print:
            print('%d)'%(i+1))
            #print(tp[0]/60, path)
            print(tp[0]/60, combine_same_trips(path))
    return refined
    
#print_answ(answ)   
############################################################
## Format Explanation
## (total_minutes, [(stop1, departure_time, trip_id, trip_duration), (stop2, departure_time, ...) ... ])
############################################################

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def print_human_readable(routes):
    outstring = u'\n#################################################################\n\n'
    for route in routes:
        trip_time = math.ceil(route[0])
        outstring += u'Trip Time: {}\n'.format(trip_time)
        outstring += u'-----------\n'
        outstring += u'|Directions|\n'
        outstring += u'------------\n\n'
        outstring += u'---------------------------------------------------------------------------------------------------------------\n'
        outstring += u'|{:>40}|{:>24}|{:>12}|{:>30}|\n'.format('Stop Name', 'Departure Time', 'Duration', 'Trip Name')
        outstring += u'---------------------------------------------------------------------------------------------------------------\n'
        for item in route[1]:
            outstring += u'|{:>40}|{:>24}|{:>12}|{:>30}|\n'.format(stops.loc[item[0]][u'stop_name'], item[1], item[3], item[2])
        outstring += u'---------------------------------------------------------------------------------------------------------------\n'
        outstring +=u'\n#################################################################\n'
        
    print(outstring)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Evaluation

## Test I

These ground truth examples for testing were given by the TA. All tests in this section are run with k shortest not using the probability treshold!


start =  Zürich HB (8503000) to Zürich

end = Auzelg (8591049)

arrival = '12:30:00'

Possible Solutions:

<b>Route 1:</b>

20.TA.26-9-A-j19-1.2.H: 8503000:0:41/42 at 12:07:00 ~ 8503310:0:3 at 12:17:00

Walking: 8503310:0:3 ~ 8590620

168.TA.26-12-A-j19-1.2.H: 8590620 at 12:23:00 ~ 8591049 at 12:29:00


<b>Route 2:</b>

32.TA.80-159-Y-j19-1.8.H: 8503000:0:5 at 12:05:00 ~ 8503006:0:6 at 12:11:00

Walking: 8503006:0:6 ~ 8580449

1914.TA.26-11-A-j19-1.27.R: 8580449 at 12:15:00 ~ 8591049 at 12:24:00

In [13]:
start_stop = u'8503000'
end_stop = u'8591049'
T = compute_seconds('12:30:00')
K = 2

# disctionary for number of shortest paths found from each node
count_u = dict()

# SBB network used for the algorithm
sbb_network = G

# all possible nodes/stations
all_stops = G.keys()

answ = k_shortest(remove_same=False)
print_human_readable(print_answ(answ[0:2], False))   

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


#################################################################

Trip Time: 23.0
-----------
|Directions|
------------

---------------------------------------------------------------------------------------------------------------
|                               Stop Name|          Departure Time|    Duration|                     Trip Name|
---------------------------------------------------------------------------------------------------------------
|                               Zürich HB|                12:07:00|    00:09:00|        20.TA.26-9-A-j19-1.2.H|
|                              Glattbrugg|                12:19:50|    00:01:09|                          walk|
|                     Glattbrugg, Bahnhof|                12:23:00|    00:06:00|      168.TA.26-12-A-j19-1.2.H|
|                          Zürich, Auzelg|                12:30:00|    00:00:00|                      Terminus|
---------------------------------------------------------------------------------------------

As you can see, result matches the ground truth test case.

## Test II

Tests in this section are manually selected and using sbb online timetable varified that they seem reasonable. Again, tests in this notebook are done without probabilities.


start =  Zürich, Witikon (8591438) to Zürich

end = Sellenburen (8503696)

arrival = '20:40:00'

In [14]:
start_stop = u'8591438'
end_stop = u'8503057'
T = compute_seconds('20:40:00')
K = 2

# disctionary for number of shortest paths found from each node
count_u = dict()

# SBB network used for the algorithm
sbb_network = G

# all possible nodes/stations
all_stops = G.keys()

answ = k_shortest(remove_same=False)
print_human_readable(print_answ(answ[0:2], False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


#################################################################

Trip Time: 65.0
-----------
|Directions|
------------

---------------------------------------------------------------------------------------------------------------
|                               Stop Name|          Departure Time|    Duration|                     Trip Name|
---------------------------------------------------------------------------------------------------------------
|                 Zürich, Witikon Zentrum|                19:35:12|    00:02:47|                          walk|
|          Zürich, Carl-Spitteler-Strasse|                19:40:00|    00:04:00|       171.TA.26-701-j19-1.5.R|
|                       Zürich, Klusplatz|                19:47:00|    00:02:00|      1284.TA.26-8-C-j19-1.8.R|
|                        Zürich, Römerhof|                19:51:00|    00:08:00|      1484.TA.26-3-A-j19-1.8.R|
|                 Zürich, Bahnhofplatz/HB|                20:02:15|    00:00:44|             

Test 2

In [15]:
start_stop = u'8591438'
end_stop = u'8591080'
T = compute_seconds('08:16:00')
K = 2

# disctionary for number of shortest paths found from each node
count_u = dict()

# SBB network used for the algorithm
sbb_network = G

# all possible nodes/stations
all_stops = G.keys()

answ = k_shortest(remove_same=False)
print_human_readable(print_answ(answ[0:2], False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


#################################################################

Trip Time: 41.0
-----------
|Directions|
------------

---------------------------------------------------------------------------------------------------------------
|                               Stop Name|          Departure Time|    Duration|                     Trip Name|
---------------------------------------------------------------------------------------------------------------
|                 Zürich, Witikon Zentrum|                07:35:12|    00:02:47|                          walk|
|          Zürich, Carl-Spitteler-Strasse|                07:40:00|    00:05:00|        59.TA.26-701-j19-1.3.R|
|                       Zürich, Klusplatz|                07:47:00|    00:07:00|      1366.TA.26-8-C-j19-1.8.R|
|                   Zürich Stadelhofen FB|                07:57:00|    00:04:00|    2017.TA.26-11-A-j19-1.27.R|
|                     Zürich, Bürkliplatz|                08:05:00|    00:05:00|        11.TA

Test 3

In [16]:
start_stop = u'8591269'
end_stop = u'8591173'
T = compute_seconds('10:45:00')
K = 2

# disctionary for number of shortest paths found from each node
count_u = dict()

# SBB network used for the algorithm
sbb_network = G

# all possible nodes/stations
all_stops = G.keys()

answ = k_shortest(remove_same=False)
print_human_readable(print_answ(answ[0:2], False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


#################################################################

Trip Time: 28.0
-----------
|Directions|
------------

---------------------------------------------------------------------------------------------------------------
|                               Stop Name|          Departure Time|    Duration|                     Trip Name|
---------------------------------------------------------------------------------------------------------------
|                    Zürich, Manesseplatz|                10:17:20|    00:06:39|                          walk|
|                       Zürich Giesshübel|                10:26:00|    00:05:00|        17.TA.26-4-B-j19-1.1.R|
|                           Zürich HB SZU|                10:33:15|    00:00:44|                          walk|
|                 Zürich, Bahnhofplatz/HB|                10:36:00|    00:07:00|      1554.TA.26-10-j19-1.11.R|
|                      Zürich, Haldenbach|                10:45:00|    00:00:00|             